In [5]:
import pandas as pd

import requests
import json

In [9]:
# https://medium.com/swlh/using-python-to-connect-to-stravas-api-and-analyse-your-activities-dummies-guide-5f49727aac86

import requests
import json
# Make Strava auth API call with your 
# client_code, client_secret and code

CLIENT_ID = '38959'
CLIENT_SECRET_KEY = '79fdd7f8e48be1c2afa50463d83eb27af8819815'
CODE_FROM_URL = '46313967b7a9839faf793b976c60c73084c34f8c'

response = requests.post(
                    url = 'https://www.strava.com/oauth/token',
                    data = {
                            'client_id': CLIENT_ID,
                            'client_secret': CLIENT_SECRET_KEY,
                            'code': CODE_FROM_URL,
                            'grant_type': 'authorization_code'
                            }
                )
#Save json response as a variable
strava_tokens = response.json()
# Save tokens to file
with open('strava_tokens.json', 'w') as outfile:
    json.dump(strava_tokens, outfile)
# Open JSON file and print the file contents 
# to check it's worked properly
with open('strava_tokens.json') as check:
  data = json.load(check)
print(data)

{'token_type': 'Bearer', 'expires_at': 1614732481, 'expires_in': 21600, 'refresh_token': '1be0d6de690d5e17e90f397b9fc427bb49da7251', 'access_token': 'f5b1d486cc67603b15e92132b21c527b12971d1c', 'athlete': {'id': 22636627, 'username': 'andreylevin', 'resource_state': 2, 'firstname': 'Andrey', 'lastname': 'Levin', 'bio': 'https://github.com/levinandrey/bike-routs\n', 'city': '', 'state': '', 'country': '', 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2017-06-12T14:42:16Z', 'updated_at': '2021-01-10T23:08:10Z', 'badge_type_id': 0, 'weight': 72.0, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/22636627/9563277/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/22636627/9563277/1/large.jpg', 'friend': None, 'follower': None}}


In [10]:
# Eventually Getting Your Activities!

import requests
from pandas.io.json import json_normalize
import json
import csv
# Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
# Loop through all activities
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']
# Get first page of activities from Strava with all fields
r = requests.get(url + '?access_token=' + access_token)
r = r.json()
    
df = json_normalize(r)
df.to_csv('strava_activities_all_fields.csv')

<ipython-input-10-4368709a2ea0>:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(r)


In [11]:
import pandas as pd
import requests
import json
# Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
# Loop through all activities
page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']
# Create the dataframe ready for the API call to store your activity data
activities = pd.DataFrame(
    columns = [
            "id",
            "name",
            "start_date_local",
            "type",
            "distance",
            "moving_time",
            "elapsed_time",
            "total_elevation_gain",
            "end_latlng",
            "external_id"
    ]
)
while True:
    
    # get page of activities from Strava
    r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
    r = r.json()
    
    # if no results then exit loop
    if (not r):
        break
    
    # otherwise add new data to dataframe
    for x in range(len(r)):
        activities.loc[x + (page-1)*200,'id'] = r[x]['id']
        activities.loc[x + (page-1)*200,'name'] = r[x]['name']
        activities.loc[x + (page-1)*200,'start_date_local'] = r[x]['start_date_local']
        activities.loc[x + (page-1)*200,'type'] = r[x]['type']
        activities.loc[x + (page-1)*200,'distance'] = r[x]['distance']
        activities.loc[x + (page-1)*200,'moving_time'] = r[x]['moving_time']
        activities.loc[x + (page-1)*200,'elapsed_time'] = r[x]['elapsed_time']
        activities.loc[x + (page-1)*200,'total_elevation_gain'] = r[x]['total_elevation_gain']
        activities.loc[x + (page-1)*200,'end_latlng'] = r[x]['end_latlng']
        activities.loc[x + (page-1)*200,'external_id'] = r[x]['external_id']
    # increment page
    page += 1
# Export your activities file as a csv 
# to the folder you're running this script in
activities.to_csv('strava_activities.csv')

# Automating the Retrieval of the Tokens

In [12]:
import requests
import json
import time
# Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
# If access_token has expired then 
# use the refresh_token to get the new access_token
if strava_tokens['expires_at'] < time.time():
# Make Strava auth API call with current refresh token
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': CLIENT_ID,
                                'client_secret': CLIENT_SECRET_KEY,
                                'grant_type': 'refresh_token',
                                'refresh_token': strava_tokens['refresh_token']
                                }
                    )
# Save response as json in new variable
    new_strava_tokens = response.json()
# Save new tokens to file
    with open('strava_tokens.json', 'w') as outfile:
        json.dump(new_strava_tokens, outfile)
# Use new Strava tokens from now
    strava_tokens = new_strava_tokens
# Open the new JSON file and print the file contents 
# to check it's worked properly
with open('strava_tokens.json') as check:
  data = json.load(check)
print(data)

{'token_type': 'Bearer', 'expires_at': 1614732481, 'expires_in': 21600, 'refresh_token': '1be0d6de690d5e17e90f397b9fc427bb49da7251', 'access_token': 'f5b1d486cc67603b15e92132b21c527b12971d1c', 'athlete': {'id': 22636627, 'username': 'andreylevin', 'resource_state': 2, 'firstname': 'Andrey', 'lastname': 'Levin', 'bio': 'https://github.com/levinandrey/bike-routs\n', 'city': '', 'state': '', 'country': '', 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2017-06-12T14:42:16Z', 'updated_at': '2021-01-10T23:08:10Z', 'badge_type_id': 0, 'weight': 72.0, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/22636627/9563277/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/22636627/9563277/1/large.jpg', 'friend': None, 'follower': None}}


# Token my get

In [32]:

import requests
import json
import time

def getNowToken():
    # Get the tokens from file to connect to Strava
    with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)
    # If access_token has expired then 
    # use the refresh_token to get the new access_token
    if strava_tokens['expires_at'] < time.time():
        # Make Strava auth API call with current refresh token
        response = requests.post(
                            url = 'https://www.strava.com/oauth/token',
                            data = {
                                    'client_id': CLIENT_ID,
                                    'client_secret': CLIENT_SECRET_KEY,
                                    'grant_type': 'refresh_token',
                                    'refresh_token': strava_tokens['refresh_token']
                                    }
                        )
        # Save response as json in new variable
        new_strava_tokens = response.json()
        # Save new tokens to file
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
        # Use new Strava tokens from now
        strava_tokens = new_strava_tokens
        # Open the new JSON file and print the file contents 
        # to check it's worked properly
    
    return strava_tokens['access_token']
    
    #with open('strava_tokens.json') as check:
    #  data = json.load(check)
    #print(data)


nowToken = ''

nowToken = getNowToken()
print(nowToken)


f5b1d486cc67603b15e92132b21c527b12971d1c


# My expiriense List Segment Efforts (getEffortsBySegmentId)

In [27]:
#from __future__ import print_statement
import time
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint

# Configure OAuth2 access token for authorization: strava_oauth
swagger_client.configuration.access_token = nowToken

# create an instance of the API class
api_instance = swagger_client.SegmentEffortsApi()
segmentId = 56 # Integer | The identifier of the segment.
startDateLocal = '2013-10-20T19:20:30+01:00' # Date | ISO 8601 formatted date time. (optional)
endDateLocal = '2013-10-20T19:20:30+01:00' # Date | ISO 8601 formatted date time. (optional)
perPage = 56 # Integer | Number of items per page. Defaults to 30. (optional) (default to 30)

try: 
    # List Segment Efforts
    api_response = api_instance.getEffortsBySegmentId(segmentId, startDateLocal=startDateLocal, endDateLocal=endDateLocal, perPage=perPage)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling SegmentEffortsApi->getEffortsBySegmentId: %s\n" % e)

ModuleNotFoundError: No module named 'swagger_client'

In [48]:
from pprint import pprint

url_strava_api = 'https://www.strava.com/api/v3/'

def dicToStringInRequest(inp):

    #print(inp)
    out = ''
    for row in inp:
        print(row, inp[row])
        out += str(row) + '=' + str(inp[row]) + '&'
    out = out[:-1]
    #print(out)
    
    return out


def getSegments():
    url_segments_explore = 'segments/explore'
    
    data_segments_explore = {
        'access_token': nowToken,
        'bounds': '',
        'activity_type': 'riding',
        'min_cat': '',
        'max_cat': '',  
    }
    
    params = dicToStringInRequest(data_segments_explore)
    url_to_resp = url_strava_api + url_segments_explore + '?' + params
    print(url_to_resp)
    
    r = requests.get(url_to_resp)
    r = r.json()

    print('RESPONSE')
    pprint(r)
    
    
    print('\n', 'Request', 'activities')
    url = "https://www.strava.com/api/v3/activities"
    access_token = nowToken
    # Get first page of activities from Strava with all fields
    r = requests.get(url + '?access_token=' + access_token)
    r = r.json()
    df = json_normalize(r)
    pprint(df)

getSegments()


access_token f5b1d486cc67603b15e92132b21c527b12971d1c
bounds 
activity_type riding
min_cat 
max_cat 
https://www.strava.com/api/v3/segments/explore?access_token=f5b1d486cc67603b15e92132b21c527b12971d1c&bounds=&activity_type=riding&min_cat=&max_cat=
RESPONSE
{'errors': [{'code': 'invalid', 'field': 'bounds', 'resource': 'Leaderboard'}],
 'message': 'Bad Request'}

 Request activities


<ipython-input-48-b28ff33e77fc>:46: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(r)


    resource_state                                               name  \
0                2  Ночной велозаезд в магазин и обратно по Шипило...   
1                2                                       Вдоль канала   
2                2         По снежку от МЦК до Коломенской через мост   
3                2                        Левобережная - Долгопрудный   
4                2                                  Дневной велозаезд   
5                2                            От Коломенской по парку   
6                2                          Вдоль Шипиловской плотины   
7                2                           В той дом через Царицыно   
8                2                                            Морозно   
9                2                                      Посетить икею   
10               2            Ночной велозаезд до мака через Царицыно   
11               2           Ночной велозаезд через Царицыно до мкада   
12               2                         До совхо